std::thread::hardware_concurrency() 在新版C++标准库中是一个很有用的函数。返回值也仅仅是一个提示，当系统信息无法获取时，函数也会返回0。

### 原生并行版的 std::accumulate

In [1]:
#include <iostream>
#include <thread>
#include <algorithm>
#include <vector>
#include <numeric> // std::accumulate

In [2]:
namespace n1 {
    template<typename Iterator, typename T>
    struct accumulate_block
    {
        void operator()(Iterator first, Iterator last, T& result)
        {
            result = std::accumulate(first, last, result);
        }
    };

    template<typename Iterator, typename T>
    T parallel_accumulate(Iterator first, Iterator last, T init)
    {
        unsigned long const length = std::distance(first, last);
        if (!length) // 1
            return init;
        unsigned long const min_per_thread = 25;
        unsigned long const max_threads =
            (length + min_per_thread - 1) / min_per_thread; // 2
        unsigned long const hardware_threads = std::thread::hardware_concurrency();
        unsigned long const num_threads = std::min(hardware_threads != 0 ? hardware_threads : 2, max_threads);
        unsigned long const block_size = length / num_threads; // 4
        std::vector<T> results(num_threads);
        std::vector<std::thread> threads(num_threads - 1); // 5
        Iterator block_start = first;
        for (unsigned long i = 0; i < (num_threads - 1); ++i)
        {
            Iterator block_end = block_start;
            std::advance(block_end, block_size); // 6
            threads[i] = std::thread( // 7
                accumulate_block<Iterator, T>(),
                block_start, block_end, std::ref(results[i]));
            block_start = block_end; // 8
        }
        accumulate_block<Iterator, T>()(
            block_start, last, results[num_threads - 1]); // 9
        std::for_each(threads.begin(), threads.end(), std::mem_fn(&std::thread::join)); // 10
        return std::accumulate(results.begin(), results.end(), init); // 11
    }
}

In [3]:
{
    using namespace n1;
    
    std::vector<int> v;
    for (int i = 0; i < 1000; ++i) {
        v.push_back(i);
    }
    int sum = parallel_accumulate(v.begin(), v.end(), 0);
    std::cout << sum << "\n";
}

499500


结束这个例子之前，需要明确：
- T类型的加法运算不满足结合律(比如，对于float型或double型，在进行加法操作时，系统很可能会做截断操作)，因为对范围中元素的分组，会导致
parallel_accumulate得到的结果可能与 std::accumulate 得到的结果不同。
- 同样的，这里对迭代器的要求更加严格：
- 必须都是向前迭代器，而 std::accumulate 可以在只传入迭代器的情况下工作。
- 对于创建出results容器，需要保证T有默认构造函数。
- 对于算法并行，通常都要这样的修改；不过，需要根据算法本身的特性，选择不同的并行方式。

需要注意的：因为不能直接从一个线程中返回一个值，所以需要传递results容器的引用到线程中去。另一个办法，通过地址来获取线程执行的结果；